In [3]:
import torch
low_rank_grad = torch.load("low_rank_grad.pt")
unproj = torch.load("galore_unproj.pt")
ortho_mat = torch.load("ortho_matrix.pt")
low_rank_update = torch.load("low_rank_update.pt")

dist_low_rank_grad = torch.load("dist_low_rank_grad.pt")
dist_unproj = torch.load("dist_galore_unproj.pt")
dist_ortho_mat = torch.load("dist_galore_ortho.pt")
dist_low_rank_update = torch.load("dist_low_rank_update.pt")

In [15]:
low_rank_grad[:32][0], dist_low_rank_grad[0]

(tensor([-8.5231e+00,  7.1526e-07,  4.7684e-07,  0.0000e+00,  2.3842e-07,
          5.9605e-08, -8.9407e-07, -2.9802e-07], device='cuda:0'),
 tensor([ 8.5231e+00,  5.9605e-08, -2.3842e-07, -5.3644e-07,  4.7684e-07,
          5.9605e-08,  1.7881e-07,  5.9605e-07], device='cuda:0'))

In [36]:
grad = torch.load("grad.pt").cuda()
dist_grad = torch.load("dist_grad.pt").cuda()
print(f"mean error: {(grad[:32] - dist_grad).mean()}")
U, *_ = torch.linalg.svd(grad[:32])
_U, *_ = torch.linalg.svd(dist_grad)
U == _U


mean error: 1.32749278236588e-09


tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]], device='cuda:0')

In [17]:
dist_low_rank_grad.shape

torch.Size([32])

In [92]:
(ortho_mat @ low_rank_grad   ==  dist_ortho_mat @ dist_low_rank_grad ).all()

tensor(True, device='cuda:0')

In [ ]:
import torch
import bitsandbytes.functional as F
low_rank_grad = torch.load("low_rank_grad.pt")
dist_low_rank_grad = torch.load("dist_low_rank_grad.pt")
def update(low_rank_grad):
    p = torch.zeros_like(low_rank_grad, dtype=torch.float32, device="cuda")
    p.grad = low_rank_grad
    lr = 1e-2
    state = {}
    state["state1"] = torch.zeros_like(low_rank_grad, dtype=torch.float32, device="cuda")
    state["state2"] = torch.zeros_like(low_rank_grad, dtype=torch.float32, device="cuda")
    beta1 = 0.9
    beta2 = 0.999
    step = 1
    args = None
    eps = 1e-8
    weight_decay = 1e-2


    F.optimizer_update_32bit(
        "adam",
        p.grad,
        p,
        state["state1"],
        beta1,
        eps,
        step,
        lr,
        state["state2"],
        beta2,
        weight_decay
    )
    return p
print(low_rank_grad.shape, dist_low_rank_grad.shape)
assert (low_rank_grad[:, :32] == dist_low_rank_grad).all()
# adam update step
low_rank_grad = update(low_rank_grad)
dist_low_rank_grad = update(dist_low_rank_grad)
low_rank_grad[:, :32] == dist_low_rank_grad

torch.Size([8, 32]) torch.Size([8, 32])


In [93]:
low_rank_update == dist_low_rank_update

tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False,  True,  True, False, False,
         False, False, False, False, False, False, False, False, False, False,
          True, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False,  True, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False,  True],
        [False, False, False, False, False, False, False, False, False,  True,
         False, False, False, False, False, False,  True, False, False, False,
         False, False, False, False, False, False, False, F

In [36]:
ortho_mat @ low_rank_grad

tensor([[-0.7474, -1.2540, -0.2261,  ..., -0.6056,  0.0814,  0.4249],
        [ 0.8328,  0.9763,  0.1816,  ...,  1.6249, -0.5379,  0.8654],
        [ 0.5236, -0.4164,  0.1525,  ..., -0.7056,  0.5073, -0.8464],
        ...,
        [-0.1320, -0.4498,  0.1109,  ..., -0.7294, -0.4453, -0.5510],
        [-0.4251,  0.1342, -0.1071,  ..., -0.1551,  0.0112,  0.4830],
        [ 0.0945,  0.8062, -0.0879,  ...,  0.7416, -0.0420, -0.8317]],
       device='cuda:0')

In [28]:
unproj

torch.Size([32, 32])